In [1]:
from environment.base import RealUserEnvironment, _FreeEnvironment, _GuidedEnvironment
from data.dataset import GraphDataset
from chatbot.adviser.app.answerTemplateParser import AnswerTemplateParser
from chatbot.adviser.app.logicParser import LogicTemplateParser
from chatbot.adviser.app.systemTemplateParser import SystemTemplateParser
from chatbot.adviser.app.parserValueProvider import RealValueBackend
from chatbot.adviser.app.rl.utils import AutoSkipMode 

/fs/scratch/users/vaethdk/adviser_reisekosten/.env/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
graph = GraphDataset(graph_path='resources/test_graph.json', answer_path='resources/test_answers.json', use_answer_synonyms=True)

===== Dataset Statistics =====
- files:  resources/test_graph.json resources/test_answers.json
- synonyms: True
- depth: 32  - degree: 13
- answers: 162
- questions: 173


In [3]:
answer_parser = AnswerTemplateParser()
logic_parser = LogicTemplateParser()
system_parser = SystemTemplateParser()
value_backend = RealValueBackend(graph.a1_countries)

In [4]:
env = _FreeEnvironment(
    env_id=1,
    dataset=graph,
    sys_token="[SYS]", usr_token="[USR]", sep_token="",
    max_steps=30, max_reward=30, user_patience=3,
    answer_parser=answer_parser, system_parser=system_parser, logic_parser=logic_parser,
    value_backend=value_backend,
    auto_skip=AutoSkipMode.NONE
)

In [5]:
env.reset(0)

In [6]:
env.goal_node.text

'Buchung 1. Klasse ist möglich ab einer Entfernung von mehr als 100 km.'

In [7]:
env.goal.initial_user_utterance

'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?'

In [8]:
env.step(0)

({'dialog_node': DialogNode.QUESTION(key: 16348058621438633, answers: 6, questions: 0)
          - connected_node: None
          - text: Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste a
          ,
  'dialog_node_key': 16348058621438633,
  'original_user_utterance': 'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  'current_user_utterance': 'Ich muss verreisen',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.'],
  'user_utterance_history': ['Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
   'Ich muss verreisen'],
  'dialog_history': [('[SYS]',
    'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklick

In [9]:
env.step(1)

({'dialog_node': DialogNode.QUESTION(key: 16348058588084537, answers: 3, questions: 3)
          - connected_node: None
          - text: Handelt es sich um eine Dienstreise oder einen Dienstgang?
          ,
  'dialog_node_key': 16348058588084537,
  'original_user_utterance': 'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  'current_user_utterance': 'Ich muss verreisen',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?'],
  'user_utterance_history': ['Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
   'Ich muss verreisen',
   'Ich muss verreisen'],
  'dialog_history': [('[SYS]',
    'Zu welchem Thema haben Sie Fragen?Sie können entweder eine

In [10]:
env.step(0)

({'dialog_node': DialogNode.QUESTION(key: 16348058588084537, answers: 3, questions: 3)
          - connected_node: None
          - text: Handelt es sich um eine Dienstreise oder einen Dienstgang?
          ,
  'dialog_node_key': 16348058588084537,
  'original_user_utterance': 'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  'current_user_utterance': 'Dienstreise',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?'],
  'user_utterance_history': ['Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
   'Ich muss verreisen',
   'Ich muss verreisen',
   'Dienstreise'],
  'dialog_history'

In [11]:
env.step(1)

({'dialog_node': DialogNode.QUESTION(key: 16348064261907944, answers: 2, questions: 0)
          - connected_node: None
          - text: Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?
          ,
  'dialog_node_key': 16348064261907944,
  'original_user_utterance': 'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  'current_user_utterance': 'Dienstreise',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?'],
  'user_utterance_history': ['Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  

In [20]:
env.step(1)

({'dialog_node': DialogNode.QUESTION(key: 16348064261907944, answers: 2, questions: 0)
          - connected_node: None
          - text: Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?
          ,
  'dialog_node_key': 16348064261907944,
  'original_user_utterance': 'Meine Tasche wurde gestohlen, wer hilft mir weiter?',
  'current_user_utterance': 'Notfall auf der Dienstreise',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?'],
  'user_utterance_history': ['Meine Tasche wurde gestohlen, wer hilft mir weiter?',
   'Notfall auf der Dienstreise',
   'Notfall auf der Dienstrei

In [12]:
env.step(0)

({'dialog_node': DialogNode.QUESTION(key: 16348064261907944, answers: 2, questions: 0)
          - connected_node: None
          - text: Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?
          ,
  'dialog_node_key': 16348064261907944,
  'original_user_utterance': 'Ab wann darf ich bei einer Zugfahrt die 1. Klasse nehmen?',
  'current_user_utterance': 'nicht',
  'system_utterance_history': ['Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Zu welchem Thema haben Sie Fragen?Sie können entweder eine passende Option aus der Vorschlagsliste anklicken oder unten Text eingeben.',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Handelt es sich um eine Dienstreise oder einen Dienstgang?',
   'Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?',
   'Haben Sie die schriftliche Genehmigung ihres Vorgesetzten eingeholt?'],
  'user_utterance_histo